In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from random import randint
import re

##### Its my first notebook so its basiccly just playing around with data and charts
# TABLE OF CONTENTS
## 1 Getting & Checking Data [-->](#1.-GETTING-&-CHECKING-DATA)
## 2. Overview [-->](#2.-OVERVIEW)
####     - 2.1 Preparing Data
####     - 2.2 Which Color is victorius ?
####     - 2.3 How the game usually ends ?
####     - 2.4 Usually played openings ?
####     - 2.5 How many book moves ?
## 3 GETTING SOME INTERESTING DATA [-->](#3.-GETTING-SOME-INTERESTING-DATA)
####     - 3.1 The Longest and The Shortest game
####     - 3.2 Short or Long castle ?
####     - 3.3 Longest Mate Mesh ?
####     - 3.4 Biggest Rating Diffrence ?
####     - 3.5 Distribution of mates by piece ?
####     - 3.6 How often pons are promoted ?
####     - 3.7 Whats the first move?
####     - 3.8 Exacly the same game?
####     - 3.9 How each field on the chessboard is burdened?
## 4 CONCLUSION [-->](#4.-CONCLUSION)

# 1. GETTING & CHECKING DATA

 - Loading data to the dataframe
 - Checking columns types 
 - checking for missing values 

In [2]:
df = pd.read_csv('/kaggle/input/chess/games.csv')

In [3]:
df.isnull().sum()

#### There is no missing values, so we can proceed
#### Lets check if values in columns are saved in proper type 


In [4]:
df.dtypes

#### Looks good, but just to be sure lets return random row to check it for sure

In [5]:
df.iloc[randint(0,len(df))]

#### Everything looks OK, now we can analyze the dataset

# 2. OVERVIEW

## 2.1 Preparing data
### Lets create the new column which will contain the mean of rating of two players 

In [6]:
df['avg_rating'] = (df['white_rating']+df['black_rating'])/2
df['avg_rating'].describe()['min':'max']

### as we can see min-max rating is 816-2475 so we can assign any game to some rating range and create new categorical column containing game level

In [7]:
r = (2500-800)/4
print(f'Lets say we have 4 levels \n')
lev_tab = ['low', 'mid', 'high', 'pro']
for x in range(0,4):
    print(lev_tab[x] + ' - ' + str(int(800+r*x)) + '-' + str(int(800+r*(x+1))))

In [8]:
def fun(rat):
    for x in range(0,4):
        if rat >= 800+r*x and rat < 800+r*(x+1):
            return lev_tab[x]

df['level'] = df['avg_rating'].apply(fun)
df['level'].value_counts()

#### Okay now we can analyze diffrent aspects of datafram based on game level 
## For Example, lets get average of turns in game for every level

In [9]:
g = df.groupby(df['level']).mean()['turns'].sort_values().round()
print(g)

### as we can see the higher the level the game last longer, lets do some data visualization

In [10]:
fig = px.histogram(g, x=g.index, y=g.values, histfunc='avg', labels={'y':'moves'} ,title='Average game lenght',
                   opacity=0.6, color=g.values, color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.update_layout(bargap=0)
fig.show()

## we can also see scatter chart with avg rating value

In [11]:
fig = px.scatter(df, x='turns', y='avg_rating', color='level')
fig.show()

## 2.2 Which color is victorious ?

In [12]:
df['count'] = 1 
piv = pd.pivot_table(df, index='level',values='count', columns='winner',aggfunc='count')
df.drop(columns='count', axis=1)
piv

## Data Visualization

In [13]:
colors = ['black', 'lightgrey', 'white']

fig = make_subplots(rows=1, cols=4, specs=[[{'type':'domain'}, {'type':'domain'},{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=piv.columns, values=piv.values[1], name='low'),1,1)
fig.add_trace(go.Pie(labels=piv.columns, values=piv.values[2], name='mid'),1,2)
fig.add_trace(go.Pie(labels=piv.columns, values=piv.values[0], name='high'),1,3)
fig.add_trace(go.Pie(labels=piv.columns, values=piv.values[3], name='pro'),1,4)

fig.update_traces(hole=.4, hoverinfo='label+percent+name', textfont_size=14,
                  marker=dict(colors=colors, line=dict(color='#F2F2F2', width=2)))

fig.update_layout(title_text="Victorious Color",
    annotations=[dict(text='LOW', x=0.085, y=0.5, font_size=16, showarrow=False),
                 dict(text='MID', x=0.37, y=0.5, font_size=16, showarrow=False),
                 dict(text='HIGH', x=0.63, y=0.5, font_size=16, showarrow=False),
                 dict(text='PRO', x=0.91, y=0.5, font_size=16, showarrow=False)])
fig.show()

## 2.3 How the games usually ends ?

In [14]:
s = df['victory_status'].value_counts()
fig = px.pie(s, values=s.values, names=s.index, color_discrete_sequence=px.colors.qualitative.Pastel)

fig.show()

In [15]:
t = pd.pivot_table(df, values='count', index='level', columns='victory_status',aggfunc='count')
t

In [16]:
fig = px.bar(t, color_discrete_sequence=px.colors.sequential.Agsunset, title = 'Victory Status By The Game Rating')
fig.show()

## 2.4 Usually played Openings

In [17]:
print('Most popular Openings')
df['opening_name'].value_counts().head(5)

In [18]:
print('The most infamous Openings')
df['opening_name'].value_counts().tail(5)

In [19]:
len(df['opening_name'].value_counts())

In [20]:
df['opening_name'].value_counts().head(60)

#### as we can see there is a lot of these openings with diffrent variations, we should probalby shorten by combining diffrent variations of the same opening to e.g. lets say around maybe 50 - 150  openings
### But for now lets take the most popular openings with variations


In [21]:
df['opening_name'].value_counts().describe()

In [22]:
s = df['opening_name'].value_counts().head(30)          # Lets take the 30 most popular openings with variations
tab = '|'.join('^' + r"{}".format(x)+ '$' for x in s.index)
df_open = df.loc[df['opening_name'].str.contains(tab, regex=True)]
print(f'\nWe analyzing {round(len(df_open)/len(df)*100,2)} % of dataset and')
print(len(df_open['opening_name'].value_counts()))
print('most popular openings with variations')

In [23]:
t = pd.pivot_table(df_open, values='count', columns='winner', index='opening_name', aggfunc='count').fillna(0)
g = df_open.groupby('opening_name').sum()['count']
tab = [[y/g.values[i] for y in x] for i,x in enumerate(t.values)]

In [24]:
fig = go.Figure(data=go.Heatmap(z=tab, x=t.columns, y=t.index, colorscale='plasma'))
fig.update_layout(title='Winner by the opening with variation in %', yaxis_nticks=50, margin=dict(t=60, r=20, b=10, l=20))
fig.show()

In [25]:
g = df_open.groupby(by='opening_name').mean()['turns']
g = g.sort_values()

In [26]:
fig = px.bar(g.values, x=g.values, y=g.index, color=g.values)
fig.update_layout(title='Game lenght by the opening with variation', yaxis_nticks=50)
fig.show()

## Okay now let's merge the the openings with diffrent variations

In [27]:
df_open = df.copy()
df_open['opening_name'] = df_open['opening_name'].str.split(' ').str[0:2]
df_open['opening_name'] = df_open.opening_name.apply(lambda x: ' '.join([str(i).replace(':','') for i in x]))

print('Now we have: ')
print(len(df_open['opening_name'].value_counts()))
print('Openings\n')
print(df_open['opening_name'].value_counts())

### lets drop the openings with less than 30 games

In [28]:
s = df_open['opening_name'].value_counts()
s = s[s<30]
print(f'Dropping {len(s)} openings')
for x in s.index:
    df_open = df_open[~df_open.opening_name.str.contains(x)]

In [29]:
t = pd.pivot_table(df_open, values='turns' ,index='opening_name', columns='level', aggfunc='mean')
t

In [30]:
fig = go.Figure(data=go.Heatmap(z=t.values[0:31].round(),x=t.columns, y=t.index[0:31], colorscale="Inferno"))
fig.update_layout(title='Game Lenght by opening and level(part 1)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

In [31]:
fig = go.Figure(data=go.Heatmap(z=t.values[31:62].round(),x=t.columns, y=t.index[31:62], colorscale="Inferno"))
fig.update_layout(title='Game Lenght by opening and level (part 2)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

### as we can see we have some missing values, its because on the some game levels certain openings wasn't played at all
## Let's check who wins on the depending of the opening

In [32]:
table = pd.pivot_table(df_open, values='count' ,index='opening_name', columns='winner', aggfunc='count').fillna(0)
g = df_open.groupby(['opening_name']).sum()['count']
tab = [x/g.values[i] for i,x in enumerate(table.values)]

In [33]:
fig = go.Figure(data=go.Heatmap(z=tab[0:31],x=table.columns, y=table.index[0:31], colorscale="Electric"))
fig.update_layout(title='Winner by opening and level (part 1)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

In [34]:
fig = go.Figure(data=go.Heatmap(z=tab[31:62],x=table.columns, y=table.index[31:62], colorscale="Electric"))
fig.update_layout(title='Winner by opening (part 2)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

### lets consider the game level too

In [35]:
def fun(o):
    if o == 'white':
        return 1 
    elif o == 'draw':
        return 0
    else:
        return -1

df_open['win_points'] = df_open.winner.apply(fun)
table = pd.pivot_table(df_open, values='win_points' ,index=['opening_name'], columns='level', aggfunc='sum')
table

In [36]:
fig = go.Figure(data=go.Heatmap(z=table.values[0:31],x=table.columns, y=table.index[0:31], colorscale="Hot"))
fig.update_layout(title='Win by opening and level(part 1)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

In [37]:
fig = go.Figure(data=go.Heatmap(z=table.values[31:62],x=table.columns, y=table.index[31:62], colorscale="Hot"))
fig.update_layout(title='Win by opening and level(part 2)', yaxis_nticks=200, margin=dict(t=80, r=200, b=0, l=200))
fig.show()

### as we can see negative values means that black is more likely to win

## 2.5 How many book moves?
### Lets check how many book moves players play in case of game rating

In [38]:
print('The average amount of book moves is:')
print(round(df['opening_ply'].mean()))
print('\n')
df['opening_ply'].value_counts().sort_index()

### Okay lets take only the games which are longer than 20 moves and games of amount of opening moves equal or less than 20

In [39]:
df_open = df.loc[(df['opening_ply'] <= 20) &  (df['turns'] >= 20),['level', 'opening_ply']]
print(f'We will consider {round((len(df_open)*100/len(df)),2)} % of data from our dataset')

### To make this accurate we have to take % of every amount of moves separetly from every game level, because amount of games on diffrent levels is much diffrent

In [40]:
slow = df_open.loc[df_open['level']=='low',['opening_ply']].value_counts()
smid = df_open.loc[df_open['level']=='mid',['opening_ply']].value_counts()
shigh = df_open.loc[df_open['level']=='high',['opening_ply']].value_counts()
spro = df_open.loc[df_open['level']=='pro',['opening_ply']].value_counts()

slow, smid, spro, shigh = slow.sort_index()*100/slow.sum(), smid.sort_index()*100/smid.sum(), spro.sort_index()*100/spro.sum(), shigh.sort_index()*100/shigh.sum()
df_open = pd.DataFrame([spro.values, shigh.values, smid.values, slow.values], columns=[str(x) for x in range(1,21)]).fillna(0)

## Data Visualization

In [41]:
fig = px.imshow(df_open, labels=dict(x="Number of Book Moves", y="Rating", color="%"),
                x=[x for x in range(1,21)], y=['pro','high','mid','low'])
fig.show()

### Okay lets look at the cummulative Heatmap, because anybody who did e.g. 10 book moves also did it less

In [42]:
slow = slow.sort_index(ascending=False).cumsum().sort_index()
smid = smid.sort_index(ascending=False).cumsum().sort_index()
shigh = shigh.sort_index(ascending=False).cumsum().sort_index()
spro = spro.sort_index(ascending=False).cumsum().sort_index()
df_open = pd.DataFrame([spro.values, shigh.values, smid.values, slow.values], columns=[str(x) for x in range(1,21)]).fillna(0)

In [43]:
fig = px.imshow(df_open, labels=dict(x="Number of Book Moves", y="Rating", color="%"),
                x=[x for x in range(1,21)], y=['pro','high','mid','low'])
fig.show()

### Lets take a look at the funnel chart, lets take firt 10 moves and another 10 seperatly to get a better view

In [44]:
y = df_open.columns.values[0:10]
fig = go.Figure()
fig.add_trace(go.Funnel(name='low',y=y, x=df_open.values[3][0:10], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='mid',y=y, x=df_open.values[2][0:10], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='high',y=y, x=df_open.values[1][0:10], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='pro',y=y, x=df_open.values[0][0:10], textinfo = 'percent total'))

fig.show()

In [45]:
y = df_open.columns.values[10:20]
fig = go.Figure()
fig.add_trace(go.Funnel(name='low',y=y, x=df_open.values[3][10:20], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='mid',y=y, x=df_open.values[2][10:20], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='high',y=y, x=df_open.values[1][10:20], textinfo = 'percent total'))
fig.add_trace(go.Funnel(name='pro',y=y, x=df_open.values[0][10:20], textinfo = 'percent total'))

fig.show()

# 3. GETTING SOME INTERESTING DATA

## 3.1 Getting The Shortest and The Longest Game 

In [46]:
print(int(df.loc[df['victory_status'] == 'mate'].describe().min()['turns']))
print(df['turns'].max())

#### The result its absoulutly correct, anyone who play chess knows that 4 is the min amount of moves in the game of chess, it happen when white player loses by mate by black queen on h4 ( Qh4# ) - its called Fool's Mate and it goes like this: g4 e5(or e6) f3(or f4) Qh4#

### Lets check some info about the longest game, because I am really curious about the opening and couple more details of this game

In [47]:
df.iloc[[df['turns'].idxmax()],[4,5,6,7,9,11,14,15]]

In [48]:
print(df.iloc[df['turns'].idxmax()]['moves'])

### Its a game on the really high level with no bonus time per move, but unfortunetly the real game ended after 100 moves when on the board left 2 diffrent rooks and 3 pons, players started doing random moves probalby from lack of timet trying to win with the clock, lets then check another one out out  

In [49]:
df['turns'].sort_values(ascending=False).head(10)

In [50]:
df.iloc[16386,[4,5,6,7,9,11,14,15]]

In [51]:
df.iloc[16386]['moves']

### from 160-259  move white is trying to set a mate with rook and king - and the game ended by draw beacause it was 50 turns without pon movement or any figure capture

In [52]:
df.iloc[4236]['moves'] #  Same situation 

In [53]:
df.iloc[16645]['moves'] #  Same situation

In [54]:
df.iloc[15478]['moves'] # Same situation

In [55]:
df.iloc[6776]['moves'] # Same situation

### Here is kind of funny situation cause of pon structure,if u want check here is the link -> https://lichess.org/bbP6WYuH#121

## 3.2 Long and Short Castle popularity

### I just want to know how much often chess players doing short castle and long castle, we have to remember that each player can castle once during the game,

In [56]:
s = df['moves'].explode()
t,tab = ['O-O', 'O-O-O'], []
for x in range(0,len(t)):
    tab.append(len(s.loc[s.str.contains(t[x], regex=True)]))

tab.append(len(df)*2 - tab[0] - tab[1])
tab = [tab[0]/sum(tab), tab[1]/sum(tab), tab[2]/sum(tab)]

## Data visualization

In [57]:
fig = go.Figure(go.Sunburst(labels='% Castle None Short Long'.split(' '),
                            parents=['', '%' ,'%','Castle', 'Castle'],
                            values=[sum(tab),tab[0]+tab[1] ,tab[2], tab[0], tab[1]],
                            branchvalues='total'))

fig.update_layout(margin = dict(t=30, l=30, r=30, b=30))
fig.show()

## 3.3 Longest Mate Mesh
#### This will be kind of tricky, lets look at the moves column of random row to get a better view

In [58]:
df_mate = df.loc[df['victory_status'] == 'mate']
df_mate.iloc[randint(0,len(df_mate))]['moves']

#### We have to use regular regular expressions to get that exact data we need, unfortunetly RegExp dont support nested sets, so we cant use quantifiers to get the lenght of the mate mesh, so we will have to do it manually

#### We have to remember that:
 - check move can be done; by any figure even by the king, by a pon promotion, by a long/short castle
 - defending move; cant castle, can check king, can protomote a pon
 - mate move has the same properties as check the only diffrence is its ends by # instead of +
 
#### In chess notation we distinguish

- pon move e.g a3
- pon captures exd5
- pon promotion eg. e8=Q
- King, Queen, Rook, Bishop, kNight ; K, Q, R, B, N - eg. Qxe4 ,Qd4
- kNight from line 'a' capture e4 field - e.g. Naxe4
- Rook from field h2 captures h8 field - Rh2xh8
- short and long castle; O-O, O-O-O
- mate - #  - e.g. Qh4#
- check - + - e.g. Re1+
 
#### Patterns in RegExp ( lot of ways to do it ) 
  - CHECK - [^\s+]+[+] 
  - DEFE - [^\s-]+
  - MATE - [^\s+]+#$
 #### Let assume that we check maximum 40 lenght of mate mesh, i am truly convinced that nobody achieved such a long matemesh
  

In [59]:
check, defe, mate, gap = '[^\s+]+[+]', '[^\s-]+', '[^\s+]+#$', '\s'
tab = []
for m in range(0,40):
    tab.append(len(df_mate.loc[df_mate['moves'].str.contains((check+gap+defe+gap)*m+mate, regex=True)]))
print(tab)

#### okay, we got lot of zeros lets get rid of them

In [60]:
tab = [i for i in tab if i != 0]
print(len(tab))

### Okay we got it !!! 
#### The longest mate mesh including the mate move is:
## 14 

### as you have seen earlier we got 3 games lets check one of them 

In [61]:
df_mate.loc[df_mate['moves'].str.contains((check+gap+defe+gap)*13+mate, regex=True)].index

In [62]:
df.iloc[575]['moves']

### Looks like a legit game, we got pon promotion and mate mesh only by the queen
#### but lets check another game, because I want to know if situation looks simillar or if more figures are involved during the mate mesh

In [63]:
df.iloc[1377]['moves']

### okay here we have pon involved, then rook, then pon gets promoted and checks the king and finally rook and queen mates the king

### ok that went well, lets get back to our tab, fix it and load it to DataFrame, to get the matemesh distribution 

In [64]:
d = {'lenght_of_mate_mesh' : [x for x in range(1,15)], 'broken_data': tab}
matemesh_df = pd.DataFrame(data=d)
matemesh_df = matemesh_df.sort_values(by='lenght_of_mate_mesh', ascending=False)

matemesh_df['how_many_games'] = matemesh_df['broken_data'].diff().fillna(3.0)
matemesh_df = matemesh_df.astype({'how_many_games': int})
matemesh_df = matemesh_df.drop('broken_data', axis=1)
print(matemesh_df)

## Data Visualization

In [65]:
ax = sns.set_theme(style='whitegrid')
plt.figure(figsize=(20,6))
ax = sns.barplot(x='how_many_games', y='lenght_of_mate_mesh', data=matemesh_df, orient='h', palette='gist_heat_r')

#### okay nice thats look kind of like logarithimic distribution, lets turn y axis to logarithmic scale

### Data Visualziation - Logarithmic Distribution

In [66]:
plt.figure(figsize=(10,6))
ax = sns.set_theme(style='darkgrid')
ax = sns.barplot(x='lenght_of_mate_mesh', y='how_many_games', data=matemesh_df, palette='rocket_r')
ax.set_yscale('log')

## 3.4 Biggest Rating Diffrence 
#### Here i want to get the biggest rating diffrence when the player with less rating won

In [67]:
df_mate['rank'] = df_mate['white_rating'] - df_mate['black_rating']
white = df_mate.loc[(df_mate['winner'] == 'white') & (df_mate['rank'] < 0)]['rank'].min()
black = df_mate.loc[(df_mate['winner'] == 'black') & (df_mate['rank'] > 0)]['rank'].max()

if abs(white)>black: 
    print(f'Biggest Rating Diffrence is: {abs(white)}')
else:
    print(f'Biggest Rating Diffrence is: {black}')

## 3.5 Distribution of mates by piece
### I am just really currious which pieces doing mate the most often

In [68]:
df_mate = df.loc[df['victory_status'] == 'mate']
pieces = 'queen king rook bishop knight pon pon_promotion short_castle long_castle'.split(' ')
regexp_tab = ['Q.{2,3}', 'K.{2,3}', 'R.{2,5}', 'B.{2,4}', 'N.{2,5}', '[a-x]{0,2}[a-h][1-8]', '[a-x]{0,2}[a-h][1-8]=[NBRQ]', 'O-O', 'O-O-O']
regexp_tab = ['\s'+i+'#$' for i in regexp_tab]
tab = []
for x in range(0,len(regexp_tab)):
    tab.append(len(df_mate.loc[df_mate['moves'].str.contains(regexp_tab[x],regex=True)]))
    
p = pd.Series(tab, index=pieces).sort_values(ascending=False)
print(p)
if p.sum()-len(df_mate) == 0: print('The Data is Accurate')

### okay here we can notice that nobody mated with castle which is kind of sad, because thats swag in chess in my subjective opinion. Okay then we can drop the castles

In [69]:
p = p.drop(['short_castle', 'long_castle'])

## Data Visualization

In [70]:
ax = sns.set_theme(style='darkgrid')
plt.figure(figsize=(20,6))
ax = sns.barplot(x=p.index ,y=p.values,palette='Set2')

## 3.6 How often pons are promoted?

In [71]:
num = round(1/(len(df.loc[df['moves'].str.contains('=')])/len(df)))
print(f'We can say that on average in every {num} chess match pon its promoted')

## 3.7 Whats the first move ?


In [72]:
df_move = df.loc[df['moves'] != '']                  # lets take only games with at least one move
s = df_move['moves'].str.split(' ').explode()        # create new series equals to moves column, switch type to list and explode
s = s[~s.index.duplicated(keep='first')]             # drop the duplicate indexes of series 
s_per = s.value_counts()*100/len(df_move)            # First move in %
print(s_per.round(2))

### We have to preprocess the data because it doesn't look so well, lets combine moves which has less than 5 % 

In [73]:
s1 = s_per.loc[s_per.values>5]
smiss = pd.Series(s_per.loc[s_per.values<=5].sum(),index=['Other move'])
s1 = s1.append(smiss)
s1.round(2)

## Data Visualization

In [74]:
fig = px.pie(s1, values=s1.values, names=s1.index, color_discrete_sequence=px.colors.qualitative.Antique)
fig.update_traces(textinfo='percent', textfont_size=20,pull=[0,0,0.3])
fig.show()

### We can also get another 3 most popular moves and add it to the plot 

In [75]:
s2 = s_per.loc[s_per<5].head(3)
print(s2.round(2))

In [76]:
fig = go.Figure(go.Sunburst(labels='First_Move E4 D4 Other Nf3 C4 E3'.split(' '),
                            parents=['','First_Move', 'First_Move', 'First_Move', 'Other','Other', 'Other'],
                            values=[100, s1[0], s1[1], s1[2], s2[0], s2[1], s2[2]],
                            branchvalues='total',marker=dict(colorscale='Agsunset')))

fig.update_layout(margin = dict(t=30, l=30, r=30, b=30),
                  paper_bgcolor='#edd9a3')

fig.show()

## 3.8 Exacly the same game?
#### lets take only the games with more than 10 turns to avoid something like Wayward Queen Attack or Fool's Mate

In [77]:
df_new = df.drop_duplicates()
df_new = df_new.loc[df_new['turns'] >= 10]
print('The most popular game over 10 turns goes like this:')
print(df_new['moves'].value_counts().idxmax())
print('\nand it happens')
print(df_new['moves'].value_counts().max())
print('times')

### Its Italian Game: Two Knights Defense, Fried Liver Attack
#### but ended by a quick mate, because of black mistake

## 3.9 How each field on the chessboard is burdened?
### except castles of course, because thats two moves instead of one 

In [78]:
reg_end = '([=QNBR]{0}$|[=QNBR]{2}$|[+#]{1}$)'
s = df['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

## Data Visualization 

In [79]:
data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']

fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='turbid_r')
fig.update_layout(margin=dict(t=30,r=220,b=10,l=220))
fig.show()

### Okay now lets have a look how it looks like only on the Pro level

In [80]:
s = df.loc[df['level'] == 'pro']['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']
fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='turbid_r')
fig.update_layout(margin=dict(t=30,r=220,b=10,l=220))
fig.show()

### Okay we can also do it for specific Opening, let's take for example 
## French Defense

In [81]:
s = df.loc[df['opening_name'].str.contains('French Defense')]['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']
fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='Bluyl_r')
fig.update_layout(margin=dict(t=40,r=220,b=20,l=220))
fig.show()

## Sicilian Defense

In [82]:
s = df.loc[df['opening_name'].str.contains('Sicilian Defense')]['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']
fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='matter_r')
fig.update_layout(margin=dict(t=40,r=220,b=10,l=220))
fig.show()

## Caro-Kann Defense

In [83]:
s = df.loc[df['opening_name'].str.contains('Caro-Kann Defense')]['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']
fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='gray')
fig.update_layout(margin=dict(t=40,r=220,b=10,l=220))
fig.show()

## Queen's Pawn Game

In [84]:
s = df.loc[df['opening_name'].str.contains("Queen's Pawn Game")]['moves'].explode()
s = s.str.extract('([a-h][1-8])'+reg_end)[0]
s = s.value_counts().sort_index()
s = round(s/s.max(),2)
print('The Most popular fields on the chessboard :')
print(s.sort_values(ascending=False).head(5).index)

data = np.array([s.values[0:8], s.values[8:16], s.values[16:24], s.values[24:32], s.values[32:40], s.values[40:48], s.values[48:56], s.values[56:64]])
z_text = [[y+x for y in 'abcdefgh'] for x in '12345678']
fig = ff.create_annotated_heatmap(z=data.transpose() ,annotation_text=z_text, colorscale='Purpor_r')
fig.update_layout(margin=dict(t=40,r=220,b=10,l=220))
fig.show()

# 4. CONCLUSION

### Game lenght
- The average game lenght is 60 turns
- The higher the rating of the players the game last longer ( linear dependence )
- The game lasts the longest with Robatsh (modern) Defense and English Defense
- The game lasts the shortest with Semi-Slav Defense and Danish Gambit
- The game of chess doesn't usually last more than 100 moves

### Winner
- White color is slightly more likely to win, but when the rating is high draws occurs more often
- Best win rate for white has Nimzowitsh Defense, Rat Defense and Elephant Gambit
- Best win rate for black has Grob Opening and Van't Krujis Opening
- For the mid ranked players best win rate for white has Philidor Defense, although for black Van't Krujis Opening
- For the high ranked players best win rate for white has English Opening and Indian Game for black
- For the pro players white has best win rate with English Opening, black although with Modern Defense

### Opening
- The most popular opening is Opening and Sicilian Defense and Van't Kruijs Opening


### Other Info
- Higher rated players usually resign more often 
- The average amount of book moves in opening plays is : 5
- Short castle is 4.5 more popular than long
- The longest mate mesh in this dataset is 14, frequency is inversly proportional to the lenght of mate mesh and its has logarithmic dependency
- Mate by castle is really rare and usually ain't happen
- We can say that on average in every 8 chess game pon its promoted 
- The most popular field on the chessboard is f7
- The most popular field on the chessboard for Sicilian Defense is d4